<a href="https://colab.research.google.com/github/rlatpwls7337/git_test/blob/main/Lecture1_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow

In [ ]:
import numpy as np
import os,matplotlib
from matplotlib import pyplot as plt

### tensorflow 2.0 ### 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras import Input
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import losses
from tensorflow.keras import optimizers
from tensorflow.keras import metrics
from tensorflow.keras import regularizers
from tensorflow.keras import utils

In [ ]:
import tensorflow as tf
tf.config.list_physical_devices()

### 2. 데이터 로딩

In [ ]:
(x_train,y_train),(x_test,y_test) =  tf.keras.datasets.mnist.load_data(path='minist.npz')
print(x_train.shape,y_train.shape)

In [ ]:
# train과 test 시간 줄이기 위해서 학습 데이터 : 1000개, test 데이터: 200개 
## 데이터 전처리
x_train_list = []
x_test_list = []

for i, i_ in enumerate(x_train[:1000]):
    arr = np.zeros(shape=(32,32))  # shape : max pooling을 해도 에러가 안 생기도록 2의 배수로 설정(pooling 5번 수행 가능) 
    arr[:28,:28] = x_train[i]
    x_train_list.append(arr)

for i, i_ in enumerate(x_test[:200]):
    arr = np.zeros(shape=(32,32))  # shape : max pooling을 해도 에러가 안 생기도록 2의 배수로 설정(pooling 5번 수행 가능) 
    arr[:28,:28] = x_test[i]
    x_test_list.append(arr) 

x_train1 = np.expand_dims(np.array(x_train_list),axis=-1)
x_test1 = np.expand_dims(np.array(x_test_list),axis=-1)
print(x_train1.shape,x_test1.shape)

In [ ]:
y_train_list = []
y_test_list = []

for i, i_ in enumerate(y_train[:1000]):
    zero = [0]*10
    zero[i_]=1
    y_train_list.append(zero)

for i, i_ in enumerate(y_test[:200]):
    zero = [0]*10
    zero[i_]=1
    y_test_list.append(zero)

y_train1 = np.array(y_train_list)
y_test1 = np.array(y_test_list)
print(y_train1.shape,y_test1.shape)

In [ ]:
plt.figure(figsize=(10,10))

for i in range(3):
    plt.subplot(1,3,i+1)  # 3개의 그림 그리기
    plt.imshow(x_train1[i][...,0],cmap='gray')  # cmap = 'rgb'
    plt.title('Class = {}'.format(y_train[i])) 

### 3. 모델 만들기

## AI 모델을 구성하는 레이어 만들기

##### AI 모델은 여러 개의 레이어를 쌓아 올려 만듭니다.
##### 가장 대표적인 모델인 CONV-BN-ACT-POOL 구조를 만들어 보겠습니다.

##### 먼저 데이터가 들어가는 첫 번째 레이어를 만들어 봅시다.

In [ ]:
first_layer = Input(shape=(32,32,1)) # 전처리했을 때 shape와 동일 

##### 그 다음으로 데이터의 특징을 추출한 Convolusion layer을 연결하겠습니다.

In [ ]:
second_layer = layers.Conv2D(filters=8,kernel_size=(3,3),activation=None,padding='same')(first_layer)

##### 레이어 중간에서 정규화를 도와줄 Batch Normalization 과정을 추가
- 기울기 소실 문제 방지
- 데이터의 분포가 한 쪽으로 치우치지 않게 한다.
- x data의 분포를 학습이 잘 되는 지점(선형과 비선형인 부분 중간 지점)으로 이동시킴

In [ ]:
third_layer = layers.BatchNormalization()(second_layer)

##### 배치 정규화 이후 신호를 변환하여 다음 뉴런으로 전달하는 Activation function layer를 추가

In [ ]:
fourth_layer = layers.Activation('relu')(third_layer)

##### 다음으로 이미지 사이즈를 줄여주는 Pooling layer를 연결한다.

In [ ]:
fifth_layer = layers.MaxPool2D(strides=(2,2))(fourth_layer)

##### 그 후 모든 뉴런을 1차원으로 shape을 바꿔준다.

In [ ]:
sixth_layer = layers.Flatten()(fifth_layer)

##### full-connected 층에 이를 전달한다.

In [ ]:
seventh_layer = layers.Dense(100,activation='relu')(sixth_layer)

##### dropout을 사용해 일부 뉴런들을 무작위로 전원 끔

In [ ]:
eighth_layer = layers.Dropout(0.25)(seventh_layer)

##### 마지막으로 최종 결과물을 출력해주는 layer를 만들어준다.

In [ ]:
final_layer = layers.Dense(10,activation='sigmoid')(eighth_layer)

#### 지금까지 넣어준 레이어를 모델 함수에 넣어 연결하면 모델이 완성된다.

In [ ]:
model = Model(first_layer,final_layer)
model.summary()

In [ ]:
model.compile(loss=losses.CategoricalCrossentropy(),optimizer = optimizers.Adam(lr=1e-4),metrics=['accuracy']) # 학습률 = 0.0001

### 4. 모델 학습하기

In [ ]:
history = model.fit(x_train1,y_train1,epochs=20,batch_size=32,validation_data = (x_test1,y_test1),shuffle=True) # shuffle : 배치를 섞는지 여부 

### 5. 결과 확인하기

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

In [ ]:
epochs = range(1,len(acc)+1)

##### 정확도와 손실함수 그래프 그리기

In [ ]:
plt.plot(epochs,acc,'b',color='blue',label='Training ACC')
plt.plot(epochs,val_acc,'b',color='red',label='Validation ACC')
plt.title('Training and Validation Accuracy',color='w')
plt.legend()

plt.figure()

plt.plot(epochs,loss,'b',color='blue',label='Training loss')
plt.plot(epochs,val_loss,'b',color='red',label='Validation loss')
plt.title('Training and Validation Loss',color='w')
plt.legend()

plt.show()